In [31]:
from mongodb_lib import *
import yaml
import pandas as pd
from collections import defaultdict
import torch
from transformers import BertTokenizer, BertModel

In [9]:
config_infra = yaml.load(open("../infra-config-pipeline.yaml"), Loader=yaml.FullLoader)
db, fs, client = connect_to_mongodb(config_infra)

INFO:root:Successfully connected to MongoDB.


In [25]:
# Import taxonomy

taxonomy = pd.read_excel("../Categories.xlsx")
taxonomy = taxonomy[["Sub-category", "Description & Keywords.1"]]

taxonomy_json = defaultdict(lambda: defaultdict(list))

for key, value in zip(taxonomy["Sub-category"], taxonomy["Description & Keywords.1"]):

    entries = value.split("\n")
    assert len(entries) == 2

    taxonomy_json[key]["description"] = entries[0].replace("- Description: ","")
    taxonomy_json[key]["keywords"] = entries[1].replace("- Keywords: ","")

In [27]:
data = read_object(fs, "product_textual_lang_summarized")
data = pd.DataFrame(data)
data.fillna("", inplace=True)
data = data[["pdt_product_detail_PRODUCTDESCRIPTION_SUMMARIZED"]]
data.columns = ["description"]


In [30]:
texts = data['description'].tolist()

In [28]:
# Load pre-trained BERT model and tokenizer
model_name = 'bert-base-uncased'  # You can choose other variants as well
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)

/home/fabio/Desktop/data-exploration/venv/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [32]:
# Tokenize and encode texts
encoded_inputs = tokenizer(texts, padding=True, truncation=True, return_tensors='pt')

# Generate BERT embeddings
with torch.no_grad():
    model_output = model(**encoded_inputs)
    embeddings = model_output.last_hidden_state[:, 0, :]

: 

In [ ]:
# Load pre-trained BERT model and tokenizer
model_name = 'bert-base-uncased'  # You can choose other variants as well
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)

In [ ]:
encoded_inputs = tokenizer(texts, padding=True, truncation=True, return_tensors='pt')

# Generate BERT embeddings
with torch.no_grad():
    model_output = model(**encoded_inputs)
    embeddings = model_output.last_hidden_state[:, 0, :]

In [ ]:
import torch
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Convert taxonomy keywords to embeddings (average of word embeddings)
taxonomy_embeddings = {}
for category, info in taxonomy.items():
    keywords = info['keywords']
    keyword_embeddings = []
    for keyword in keywords.split(', '):
        token_ids = tokenizer.encode(keyword, add_special_tokens=False, return_tensors='pt')
        with torch.no_grad():
            keyword_output = model(token_ids)[0]
        keyword_embedding = torch.mean(keyword_output, dim=1)
        keyword_embeddings.append(keyword_embedding)
    taxonomy_embeddings[category] = torch.mean(torch.cat(keyword_embeddings, dim=0), dim=0).numpy()

# Compute cosine similarity between each text embedding and each taxonomy category embedding
similarities = cosine_similarity(embeddings.numpy(), np.array(list(taxonomy_embeddings.values())))

# Threshold for classification
threshold = 0.5

# Classify each text
classified_categories = []
for i, text in enumerate(texts):
    categories = []
    for j, (category, _) in enumerate(taxonomy.items()):
        if similarities[i][j] > threshold:
            categories.append(category)
    classified_categories.append(categories)

# Add classified categories to DataFrame
data['classified_categories'] = classified_categories
print(data)